# Proyek Analisis Data: E-Commerce Public Dataset
- **Nama:** Wildan Rahmantya Andalaluna
- **Email:** wildanandalaluna@gmail.com
- **ID Dicoding:** wra305

## Menentukan Pertanyaan Bisnis

- Produk Apa Saja yang Memiliki Pertumbuhan Penjualan Tercepat?
- Bagaimana performa penjualan pada platform E-Commerce ?
- Apakah ada tren penjualan yang lebih tinggi pada hari tertentu atau waktu tertentu?
- Di mana lokasi geografis dengan jumlah pelanggan terbanyak?

## Menyiapkan semua library yang dibuthkan

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import urllib
!pip install unidecode
import unidecode
import matplotlib.image as mpimg

: 

## Data Wrangling

### Gathering Data

In [ ]:
customers_df = pd.read_csv('data/customers_dataset.csv')
customers_df.head()

In [ ]:
geo_df = pd.read_csv('data/geolocation_dataset.csv')
geo_df.head()

In [ ]:
order_items = pd.read_csv('data/order_items_dataset.csv')
order_items.head()

In [ ]:
order_pay = pd.read_csv('data/order_payments_dataset.csv')
order_pay.head()

In [ ]:
order_rev = pd.read_csv('data/order_reviews_dataset.csv')
order_rev.head()

In [ ]:
orders_df = pd.read_csv('data/orders_dataset.csv')
orders_df.head()

In [ ]:
product_cat = pd.read_csv('data/product_category_name_translation.csv')
product_cat.head()

In [ ]:
products_df = pd.read_csv('data/products_dataset.csv')
products_df.head()

In [ ]:
sellers_df = pd.read_csv('data/sellers_dataset.csv')
sellers_df.head()

### Assessing Data
*Menilai Data*

In [ ]:
# Function to display info, null values, duplicate entries, and summary stats for each DataFrame
def assess_dataframe(df, name):
    print(f"\n### {name} DataFrame ###")
    print(f"\nInfo for {name}:")
    print(df.info())

    print(f"\nNull values in {name}:\n{df.isnull().sum()}")

    print(f"\nDuplicate entries in {name}: {df.duplicated().sum()}")

    print(f"\nSummary statistics for {name}:\n{df.describe(include='all')}")
    print("\n" + "-"*50 + "\n")


# List of DataFrames to assess
dataframes = {
    "Customers": customers_df,
    "Geolocation": geo_df,
    "Order Items": order_items,
    "Order Payments": order_pay,
    "Order Reviews": order_rev,
    "Orders": orders_df,
    "Product Categories": product_cat,
    "Products": products_df,
    "Sellers": sellers_df
}

# Loop through and assess each DataFrame
for name, df in dataframes.items():
    assess_dataframe(df, name)

# Display geolocation info separately
print("\nAdditional Info for Geolocation DataFrame:")
geo_df.info()


### Cleaning Data

In [ ]:
order_rev[order_rev.review_comment_title.isna()]


In [ ]:
order_rev.review_comment_title.value_counts()


In [ ]:
order_rev[order_rev.review_comment_message.isna()]


In [ ]:
order_rev.review_comment_message.value_counts()


In [741]:
order_rev.fillna(value="no comment", inplace=True)


In [ ]:
orders_df[orders_df.order_approved_at.isna()]


In [743]:
datetime_oi = ["shipping_limit_date"]

for column in datetime_oi:
  order_items[column] = pd.to_datetime(order_items[column])

In [744]:
datetime_or = ["review_creation_date","review_answer_timestamp"]

for column in datetime_or:
  order_rev[column] = pd.to_datetime(order_rev[column])

In [745]:
datetime_oo = ["order_purchase_timestamp","order_approved_at","order_delivered_carrier_date","order_delivered_customer_date","order_estimated_delivery_date"]

for column in datetime_oo:
  orders_df[column] = pd.to_datetime(orders_df[column])


In [ ]:
order_items.info()


In [ ]:
order_rev.info()


In [ ]:
orders_df.info()


## Exploratory Data Analysis (EDA)

### Explore **customers_df**

In [ ]:
customers_df.sample(5)


In [ ]:
customers_df.describe(include='all')


In [ ]:
customers_df.customer_id.is_unique


In [ ]:
# Memeriksa nilai yang duplikat pada kolom 'customer_id' di DataFrame 'customers_df'
duplicated_customers = customers_df.customer_id.duplicated()

# Menampilkan hasil
print(duplicated_customers)


In [ ]:
customers_df.groupby(by="customer_city").customer_id.nunique().sort_values(ascending=False)


In [ ]:
customers_df.groupby(by="customer_state").customer_id.nunique().sort_values(ascending=False)


In [ ]:
order_pay.sample(5)


In [ ]:
order_pay.describe(include='all')


In [ ]:
order_pay.groupby(by="payment_type").order_id.nunique().sort_values(ascending=False)


In [ ]:
orders_df.sample(5)


In [759]:
delivery_time = orders_df["order_delivered_customer_date"] - orders_df["order_delivered_carrier_date"]
delivery_time = delivery_time.apply(lambda x: x.total_seconds())
orders_df["delivery_time"] = round(delivery_time/86400)

In [ ]:
orders_df.sample(5)


In [ ]:
orders_df.delivery_time.hist()


In [ ]:
customer_id_in_orders_df = orders_df.customer_id.values

customers_df["status"] = customers_df["customer_id"].apply(lambda x: "Active" if x in customer_id_in_orders_df else "Non Active")

customers_df.sample(5)

In [ ]:
customers_df.groupby(by="status").customer_id.count()


Merge customers_df & orders_df

In [ ]:
cust_orders_df = pd.merge(
    left=customers_df,
    right=orders_df,
    how="left",
    left_on="customer_id",
    right_on="customer_id"
)
cust_orders_df.head()


In [ ]:
cust_orders_df.groupby(by="customer_city").order_id.nunique().sort_values(ascending=False).head(10)


In [ ]:
cust_orders_df.groupby(by="customer_state").order_id.nunique().sort_values(ascending=False).head(10)


In [ ]:
cust_orders_df.groupby(by="customer_zip_code_prefix").order_id.nunique().sort_values(ascending=False).head(10)


In [ ]:
cust_orders_df.groupby(by="order_status").order_id.nunique().sort_values(ascending=False).head(10)


**Merge order_pay & order_rev**

In [ ]:
order_payrev_df = pd.merge(
    left=order_pay,
    right=order_rev,
    how="left",
    left_on="order_id",
    right_on="order_id"
)
order_payrev_df.head()


In [ ]:
order_payrev_df.groupby(by="payment_type").order_id.nunique().sort_values(ascending=False).head(10)


In [ ]:
order_payrev_df.sort_values(by="payment_value", ascending=False)

In [ ]:
order_payrev_df.groupby(by="payment_type").agg({
    "order_id": "nunique",
    "payment_value":  ["min", "max"]
})

#### Merge cust_orders_df & order_payrev

In [ ]:
customers_df = pd.merge(
    left=cust_orders_df,
    right=order_payrev_df,
    how="left",
    left_on="order_id",
    right_on="order_id"
)
customers_df.head()

### Explore order_items & sellers_df

#### Merge order_items & sellers_df

In [ ]:
item_seller_df = pd.merge(
    left=order_items,
    right=sellers_df,
    how="left",
    left_on="seller_id",
    right_on="seller_id"
)
item_seller_df.head()

In [ ]:
item_seller_df.groupby(by="seller_city").seller_id.nunique().sort_values(ascending=False).head(10)

### Explore products_df & product_cat
#### Merge products_df & product_cat

In [ ]:
product_df = pd.merge(
    left=products_df,
    right=product_cat,
    how="left",
    left_on="product_category_name",
    right_on="product_category_name"
)
product_df.head()

In [ ]:
product_df.groupby(by="product_category_name").product_id.nunique().sort_values(ascending=False).head(10)

In [ ]:
product_df.groupby(by="product_category_name_english").product_id.nunique().sort_values(ascending=False).head(10)

#### Merge item_seller_df & product_df

In [ ]:
sellers_df = pd.merge(
    left=product_df,
    right=item_seller_df,
    how="left",
    left_on="product_id",
    right_on="product_id"
)
sellers_df.head()

In [ ]:
sellers_df.sort_values(by="price", ascending=False)

In [793]:
all_data.to_csv('all_data.csv', index=False)

In [ ]:
sellers_df.groupby(by="product_category_name_english").agg({
    "order_id": "nunique",
    "price":  ["min", "max"]
})

### Explore geo_df

In [ ]:
geo_df.sample(5)

In [783]:
def pretty_string(column):
    column_space = ' '.join(column.split())
    return unidecode.unidecode(column_space.lower())

In [784]:
geo_df['geolocation_city'] = geo_df['geolocation_city'].apply(pretty_string)

In [ ]:
geo_df.groupby('geolocation_zip_code_prefix').size().sort_values(ascending=False)

In [ ]:
geo_df[geo_df['geolocation_zip_code_prefix'] == 24220].head()

### Explore All Data
#### Merge all data

Pada kasus ini saya tidak menggabungkan dataset geolocation, karena menurut saya dataset ini tidak begitu diperlukan.

In [ ]:
all_data = pd.merge(
    left=customers_df,
    right=sellers_df,
    how="left",
    left_on="order_id",
    right_on="order_id"
)
all_data.head()

In [ ]:
all_data.info()

In [ ]:
all_data.groupby(by=["customer_city", "product_category_name_english"]).agg({
    "price": "sum",
    "freight_value": "sum"
})

In [ ]:
all_data.groupby(by=["customer_state", "product_category_name_english"]).agg({
    "price": "sum",
    "freight_value": "sum"
})

In [ ]:
all_data.groupby(by="customer_state").agg({
    "order_id": "nunique",
    "payment_value": "sum"
}).sort_values(by="payment_value", ascending=False)

In [ ]:
all_data.groupby(by="product_category_name_english").agg({
    "order_id": "nunique",
    "review_score":  ["min", "max"]
})

### Convert all_data to .csv

## Visualization & Explanatory Analysis

### Pertanyaan 1 : Produk Apa Saja yang Memiliki Pertumbuhan Penjualan Terbanyak?

In [794]:
sum_order_items_df = all_data.groupby("product_category_name_english")["product_id"].count().reset_index()
sum_order_items_df = sum_order_items_df.rename(columns={"product_id": "products"})
sum_order_items_df = sum_order_items_df.sort_values(by="products", ascending=False)
sum_order_items_df = sum_order_items_df.head(10)  # Get top 10 categories

In [ ]:
plt.figure(figsize=(12, 6))
colors = sns.color_palette("viridis", n_colors=5)  # Using a different color palette

# Create a barplot for the top-selling products
sns.barplot(x="products", y="product_category_name_english", data=sum_order_items_df.head(5), palette=colors)
plt.xlabel("Number of Products Sold", fontsize=14)
plt.ylabel("Product Category", fontsize=14)
plt.title("Top 5 Products Sold", loc="center", fontsize=18)
plt.tick_params(axis='y', labelsize=12)
plt.show()

> Produk yang paling banya terjual adalah bed_bath_table.

### Pertanyaan 2 : Bagaimana performa penjualan pada platform E-Commerce ?

In [ ]:
import pandas as pd

# Display the columns to find 'order_approved_at'
print("Columns in all_data:", all_data.columns.tolist())

# If necessary, strip whitespace from column names
all_data.columns = all_data.columns.str.strip()

# Display the first few rows to check the data
print(all_data.head())

# Convert 'order_approved_at' to datetime if it exists
if 'order_approved_at' in all_data.columns:
    all_data['order_approved_at'] = pd.to_datetime(all_data['order_approved_at'], errors='coerce')

    # Check for NaT values
    print("Number of NaT values in 'order_approved_at':", all_data['order_approved_at'].isna().sum())

    # Resample the data by month
    monthly_df = all_data.resample(rule='M', on='order_approved_at').agg({
        "order_id": "nunique",
    })

    # Change the index format to Year-Month
    monthly_df.index = monthly_df.index.strftime('%B')
    monthly_df = monthly_df.reset_index()

    # Rename the columns for clarity
    monthly_df.rename(columns={
        "order_id": "order_count",
    }, inplace=True)

    # Display the result
    print(monthly_df.head())
else:
    print("'order_approved_at' does not exist in the DataFrame.")


In [797]:
monthly_df = monthly_df.sort_values('order_count').drop_duplicates('order_approved_at', keep='last').reset_index(drop=True)

In [ ]:
monthly_df.head()

monthly_df.sort_values(by='order_count')

In [799]:
month_mapping = {
    "January": 1,
    "February": 2,
    "March": 3,
    "April": 4,
    "May": 5,
    "June": 6,
    "July": 7,
    "August": 8,
    "September": 9,
    "October": 10,
    "November": 11,
    "December": 12
}

monthly_df["month_numeric"] = monthly_df["order_approved_at"].map(month_mapping)
monthly_df = monthly_df.sort_values("month_numeric")
monthly_df = monthly_df.drop("month_numeric", axis=1)

In [ ]:
plt.figure(figsize=(10, 5))
sns.barplot(
    x=monthly_df["order_approved_at"],
    y=monthly_df["order_count"],
    color="#068DA9"
)
plt.title("Number of Orders per Month (2018)", loc="center", fontsize=20)
plt.xticks(fontsize=10, rotation=25)
plt.yticks(fontsize=10)
plt.xlabel("Month", fontsize=12)
plt.ylabel("Number of Orders", fontsize=12)
plt.show()


Terlihat bahwa terjadi penurunan pada bulan September dan Oktober.Mengalami kenaikan pada bulan November.

### Pertanyaan 3 : Apakah ada tren penjualan yang lebih tinggi pada hari tertentu atau waktu tertentu?

In [801]:
# Convert 'order_approved_at' to datetime if not already in that format
all_data['order_approved_at'] = pd.to_datetime(all_data['order_approved_at'])

# Extract day of the week and hour
all_data['day_of_week'] = all_data['order_approved_at'].dt.day_name()
all_data['hour_of_day'] = all_data['order_approved_at'].dt.hour

# Count the number of orders by day of the week
daily_sales = all_data.groupby('day_of_week')['order_id'].count().reset_index()
daily_sales = daily_sales.rename(columns={"order_id": "total_orders"})
daily_sales = daily_sales[['day_of_week', 'total_orders']]

# Count the number of orders by hour of the day
hourly_sales = all_data.groupby('hour_of_day')['order_id'].count().reset_index()
hourly_sales = hourly_sales.rename(columns={"order_id": "total_orders"})
hourly_sales = hourly_sales[['hour_of_day', 'total_orders']]


In [802]:
monthly_spend_df = monthly_spend_df.sort_values('total_spend').drop_duplicates('order_approved_at', keep='last')

In [ ]:
monthly_spend_df.head()

In [ ]:
# Sort days of the week for better visualization
days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
daily_sales['day_of_week'] = pd.Categorical(daily_sales['day_of_week'], categories=days_order, ordered=True)
daily_sales = daily_sales.sort_values('day_of_week')

plt.figure(figsize=(10, 5))
sns.barplot(x='day_of_week', y='total_orders', data=daily_sales, palette='viridis')
plt.title('Total Orders by Day of the Week', fontsize=20)
plt.xlabel('Day of the Week', fontsize=14)
plt.ylabel('Total Orders', fontsize=14)
plt.xticks(rotation=45)
plt.show()


Dari Grafik Hari Selasa menjadi puncak pemesanan dengan total > 20000 pemesanan

In [ ]:
plt.figure(figsize=(10, 5))
sns.barplot(x='hour_of_day', y='total_orders', data=hourly_sales, palette='viridis')
plt.title('Total Orders by Hour of the Day', fontsize=20)
plt.xlabel('Hour of the Day', fontsize=14)
plt.ylabel('Total Orders', fontsize=14)
plt.xticks(rotation=45)
plt.show()


Dari Grafik dapat dilihat bahwa pada jam 2 pagi menjadi waktu pemesanan tertinggi

### Pertanyaan 4 : Dimana letak geografis yang memiliki customer terbanyak?

In [806]:
other_state_geolocation = geo_df.groupby(['geolocation_zip_code_prefix'])['geolocation_state'].nunique().reset_index(name='count')
other_state_geolocation[other_state_geolocation['count']>= 2].shape
max_state = geo_df.groupby(['geolocation_zip_code_prefix','geolocation_state']).size().reset_index(name='count').drop_duplicates(subset = 'geolocation_zip_code_prefix').drop('count',axis=1)

In [807]:
geolocation_silver = geo_df.groupby(['geolocation_zip_code_prefix','geolocation_city','geolocation_state'])[['geolocation_lat','geolocation_lng']].median().reset_index()
geolocation_silver = geolocation_silver.merge(max_state,on=['geolocation_zip_code_prefix','geolocation_state'],how='inner')

In [808]:
customers_silver = customers_df.merge(geolocation_silver,left_on='customer_zip_code_prefix',right_on='geolocation_zip_code_prefix',how='inner')

In [ ]:
customers_silver.head()

In [810]:
customers_silver.to_csv("geolocation.csv", index=False)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import urllib.request
import matplotlib.image as mpimg

def plot_brazil_map(data):
    # Load the map image
    brazil = mpimg.imread(urllib.request.urlopen('https://i.pinimg.com/originals/3a/0c/e1/3a0ce18b3c842748c255bc0aa445ad41.jpg'),'jpg')

    # Set the figure size
    plt.figure(figsize=(10, 10))

    # Set the bounds for the map
    extent = [-73.98283055, -33.8, -33.75116944, 5.4]

    # Create the heatmap using seaborn
    heatmap_data = data[['geolocation_lat', 'geolocation_lng']]

    # Plotting the heatmap
    ax = plt.gca()  # Get the current axis
    sns.kdeplot(data=heatmap_data, x='geolocation_lng', y='geolocation_lat', fill=True, cmap='Reds', alpha=0.5, thresh=0, ax=ax)

    # Display the base map
    plt.imshow(brazil, extent=extent, aspect='auto')

    # Set the limits to match the extent of the map image
    plt.xlim(extent[0], extent[1])
    plt.ylim(extent[2], extent[3])

    # Remove axis
    plt.axis('off')
    plt.show()

# Plotting the heatmap with unique customer data
plot_brazil_map(customers_silver.drop_duplicates(subset='customer_unique_id'))


Banyak customer yang berasalah negara bagian tenggara dan selatan.

## Conclusion

- Produk Apa Saja yang Memiliki Pertumbuhan Penjualan Tercepat?
> Hasil analisis menunjukkan bahwa produk dengan pertumbuhan penjualan tercepat adalah bed_bath_table. Meskipun terjadi penurunan pada bulan September dan Oktober, penjualan produk ini kembali meningkat secara signifikan pada bulan November.

- Bagaimana performa penjualan pada platform E-Commerce?
> Performa penjualan di platform E-Commerce menunjukkan fluktuasi yang konsisten. Dari Januari hingga Agustus, penjualan relatif stabil, dengan penurunan yang signifikan pada bulan September dan Oktober, diikuti oleh kenaikan tajam pada bulan November. Penurunan kembali terlihat pada bulan Desember.

- Apakah ada tren penjualan yang lebih tinggi pada hari tertentu atau waktu tertentu?
> Analisis terhadap data penjualan berdasarkan hari dan waktu menunjukkan bahwa hari Selasa menjadi puncak pemesanan, dengan total lebih dari 20.000 pesanan. Dari sisi waktu, jam 2 pagi merupakan waktu paling populer bagi pelanggan untuk melakukan pemesanan, yang menunjukkan kecenderungan aktivitas pembelian di luar jam kerja.

- Di mana lokasi geografis dengan jumlah pelanggan terbanyak?
> Berdasarkan visualisasi data geografis, mayoritas pelanggan berasal dari wilayah tenggara dan selatan Brasil. Kota-kota besar seperti São Paulo, Rio de Janeiro, dan Porto Alegre mencatat jumlah pelanggan tertinggi, menunjukkan bahwa wilayah perkotaan merupakan pusat utama aktivitas E-Commerce di negara ini